In [17]:
from langchain.document_loaders import PyPDFLoader

loader = PyPDFLoader("lit/combined.pdf")
pages = loader.load_and_split()

In [18]:
sources = pages

In [19]:
from dotenv import load_dotenv
load_dotenv()

from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.text_splitter import CharacterTextSplitter
from langchain.docstore.document import Document
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate

In [20]:
embeddings = OpenAIEmbeddings()
source_chunks = []
splitter = CharacterTextSplitter(separator=" ", chunk_size=2024, chunk_overlap=0)
for source in sources:
    for chunk in splitter.split_text(source.page_content):
        source_chunks.append(Document(page_content=chunk, metadata=source.metadata))

In [21]:
import chromadb
client = chromadb.PersistentClient(path="data_lit")

In [22]:
search_index = Chroma.from_documents(source_chunks, embedding=embeddings, client=client)

In [23]:
retriever = search_index.as_retriever()

# Simmilarity

In [39]:
docs = search_index.similarity_search("multi-layered nature of Ukrainian identity", k=3)
for doc in docs:
    print(str(doc.metadata["page"]) + ":", doc.page_content[:500])

612: 1060 INNA MELNYKOVSKA ET AL. 
 what marks Ukrainian identity posed a considerable challenge in identity building in 
 independent Ukraine (Barrington & Herron 2004). Apart from the ethnic identities of 
 minority groups,7 there are three Ukrainian ethnic identities that compete for the 
 status of being a Ukrainian national identity (Table 1). Each of these ethnic identities 
 is embedded in a broader set of beliefs and historical memories, as well as religious, 
 linguistic and policy preferenc
539: Karina V. Korostelina 
 environment and the person they are speaking to. Kuzio29 proposes an alter 
 native vision of national division into three main groups: reformists who are 
 oriented toward the U.S. and European experience, communists who advocate 
 an orientation to Russia, and centrists who have mixed attitudes toward both 
 the West and Russia. Different versions of Ukrainian national identity are also 
 evident in the "three Ukraines" model of Hrytsak30 that includes narrat

In [28]:
doc.metadata

{'page': 483, 'source': 'lit/combined.pdf'}

# Chat

In [36]:
question = "How does Oksana Zabushko's illuminates and portray the multi-layered nature of Ukrainian identity?"

In [38]:
template = """You are a research assistant aiding in the deep literary analysis of Zabuzhko’s wider body of work, considering aspects such as postcolonial theory, feminist theory, and literary analysis. 
Although you cannot access her texts directly, your role is to extract insights from an extensive range of literary and theoretical literature at our disposal to craft substantiated analytical perspectives. Your responses should:
- Harness detailed theoretical insights from the available literature, connecting known themes, narratives, and characters in Zabuzhko’s works 
to the theories and concepts discussed in the scholarly resources.
- Assist in the development of robust and insightful arguments and viewpoints, grounding them firmly in theories and concepts from the pertinent literature. 
To foster deep and multifaceted analytical perspectives, you should integrate examples from Zabuzhko’s known works wherever possible to illuminate the points being discussed.
- Encourage rich, critical engagement with the possible contents of Zabuzhko’s works inferred from secondary sources, always with the aim of drawing deep and thoughtful analyses grounded in the scholarly literature available.
{context}
Question: {question}
Helpful Answer:"""
QA_CHAIN_PROMPT = PromptTemplate.from_template(template)

llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.5)
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=retriever,
    chain_type_kwargs={"prompt": QA_CHAIN_PROMPT}
)
result = qa_chain({"query": question})
result["result"]

'Oksana Zabuzhko\'s works illuminate and portray the multi-layered nature of Ukrainian identity through various themes and narratives. One aspect that can be explored is the intersectionality of gender and national identity. Zabuzhko\'s exploration of gender roles and the experiences of Ukrainian women within a patriarchal society sheds light on the complexities of Ukrainian identity.\n\nIn her novel "Field Research in Ukrainian Sex," the protagonist grapples with the dichotomy of victim and executioner, where Ukrainian men are portrayed as victims and Ukrainian women carry a double burden of subjugation within a subjugated society. This portrayal highlights the oppressive dynamics within Ukrainian society and the ways in which gender intersects with national identity.\n\nFurthermore, Zabuzhko emphasizes the importance of language in shaping Ukrainian identity. In the same novel, the protagonist finds solace and a sense of belonging in the Ukrainian language. By accepting and embracing